# Notebook 3: Test the API

This notebook tests the REST API endpoints.

**IMPORTANT**: The API server must be running! Open a terminal and run:
```bash
source venv/bin/activate
python -m src.api.main
```

## Setup

In [ ]:
import requests
import json
import time
from pathlib import Path

# API base URL
API_URL = "http://localhost:8000"

print(f"API URL: {API_URL}")
print("\nMake sure the server is running!")
print("  Terminal command: python -m src.api.main")

## Test 1: Health Check

First, verify the server is running.

In [ ]:
try:
    response = requests.get(f"{API_URL}/health", timeout=5)
    
    if response.status_code == 200:
        print("✅ Server is running!\n")
        print(json.dumps(response.json(), indent=2))
    else:
        print(f"⚠️  Server responded with status {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to server!")
    print("\nMake sure the server is running:")
    print("  1. Open a terminal")
    print("  2. Run: source venv/bin/activate")
    print("  3. Run: python -m src.api.main")
except Exception as e:
    print(f"❌ Error: {e}")

## Test 2: Version Endpoint

In [ ]:
response = requests.get(f"{API_URL}/version")

print("Version info:\n")
print(json.dumps(response.json(), indent=2))

## Test 3: Create a Test File

Let's create a simple Excel file to test with.

In [ ]:
from openpyxl import Workbook

# Create test file
wb = Workbook()
ws = wb.active
ws.title = "Test Sheet"

# Add some data
ws['A1'] = 'Item'
ws['B1'] = 'Value'
ws['A2'] = 'Alpha'
ws['B2'] = 100
ws['A3'] = 'Beta'
ws['B3'] = 200
ws['A4'] = 'Total'
ws['B4'] = '=SUM(B2:B3)'

test_file = Path('/workspaces/excel-differ/tmp/api_test.xlsx')
wb.save(test_file)

print(f"✅ Created test file: {test_file}")
print(f"   Size: {test_file.stat().st_size} bytes")

## Test 4: POST /api/v1/flatten

Submit a file to be flattened.

In [ ]:
print("Uploading file to /api/v1/flatten...\n")

with open(test_file, 'rb') as f:
    files = {
        'file': ('test.xlsx', f, 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    }
    
    response = requests.post(f"{API_URL}/api/v1/flatten", files=files)

if response.status_code == 202:
    result = response.json()
    job_id = result['job_id']
    
    print("✅ Job submitted!\n")
    print(f"Job ID: {job_id}")
    print(f"Status: {result['status']}")
    
    # Save job_id for next cells
    flatten_job_id = job_id
    
else:
    print(f"❌ Request failed: {response.status_code}")
    print(response.text)
    flatten_job_id = None

## Test 5: Poll for Job Status

Check if the job is complete.

In [ ]:
if flatten_job_id:
    print(f"Polling job {flatten_job_id}...\n")
    
    for i in range(30):  # Try for 30 seconds
        time.sleep(1)
        
        response = requests.get(f"{API_URL}/api/v1/jobs/{flatten_job_id}")
        job_data = response.json()
        
        status = job_data['status']
        print(f"[{i+1}] Status: {status}", end="\r")
        
        if status == 'success':
            print("\n\n✅ Job completed successfully!\n")
            print("Result:")
            print(json.dumps(job_data['result'], indent=2))
            break
            
        elif status == 'failed':
            print("\n\n❌ Job failed!\n")
            print(f"Error: {job_data.get('error')}")
            break
    
    else:
        print("\n\n⚠️  Job still running after 30 seconds")
        print("Check status manually:")  
        print(f"  curl {API_URL}/api/v1/jobs/{flatten_job_id}")
else:
    print("⚠️  No job_id from previous step")

## Test 6: Compare Two Files

Let's create a modified version and compare them.

In [ ]:
# Create modified version
from openpyxl import load_workbook

wb = load_workbook(test_file)
ws = wb.active

# Change a value
ws['B2'] = 150  # Changed from 100

# Add a new row
ws['A5'] = 'Gamma'
ws['B5'] = 300

# Update formula
ws['B4'] = '=SUM(B2:B5)'

test_file_v2 = Path('/workspaces/excel-differ/tmp/api_test_v2.xlsx')
wb.save(test_file_v2)

print(f"✅ Created modified version: {test_file_v2}")
print("\nChanges:")
print("  - Alpha value: 100 → 150")
print("  - Added Gamma: 300")
print("  - Updated SUM formula")

## Test 7: POST /api/v1/compare

In [ ]:
print("Uploading both files to /api/v1/compare...\n")

with open(test_file, 'rb') as f1, open(test_file_v2, 'rb') as f2:
    files = {
        'file_a': ('test_v1.xlsx', f1, 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'),
        'file_b': ('test_v2.xlsx', f2, 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'),
    }
    
    response = requests.post(f"{API_URL}/api/v1/compare", files=files)

if response.status_code == 202:
    result = response.json()
    compare_job_id = result['job_id']
    
    print("✅ Comparison job submitted!\n")
    print(f"Job ID: {compare_job_id}")
    
else:
    print(f"❌ Request failed: {response.status_code}")
    print(response.text)
    compare_job_id = None

## Test 8: Get Comparison Results

In [ ]:
if compare_job_id:
    print(f"Polling comparison job {compare_job_id}...\n")
    
    for i in range(30):
        time.sleep(1)
        
        response = requests.get(f"{API_URL}/api/v1/jobs/{compare_job_id}")
        job_data = response.json()
        
        status = job_data['status']
        print(f"[{i+1}] Status: {status}", end="\r")
        
        if status == 'success':
            print("\n\n✅ Comparison completed!\n")
            
            result = job_data['result']
            
            # Show summary
            print("Summary:")
            print(json.dumps(result['summary'], indent=2))
            
            # Show some changes
            if 'diff_json' in result:
                print("\nChanges (first 10):")
                for change in result['diff_json'][:10]:
                    print(json.dumps(change, indent=2))
                    print("-" * 50)
            
            break
            
        elif status == 'failed':
            print("\n\n❌ Comparison failed!\n")
            print(f"Error: {job_data.get('error')}")
            break
    
    else:
        print("\n\n⚠️  Job still running")
else:
    print("⚠️  No job_id from previous step")

## Test 9: Test Error Handling

Try uploading an invalid file to see error response.

In [ ]:
# Create a text file (not Excel)
invalid_file = Path('/workspaces/excel-differ/tmp/not_excel.txt')
invalid_file.write_text('This is not an Excel file')

print("Testing error handling with invalid file...\n")

with open(invalid_file, 'rb') as f:
    files = {'file': ('test.txt', f, 'text/plain')}
    response = requests.post(f"{API_URL}/api/v1/flatten", files=files)

print(f"Status code: {response.status_code}")

if response.status_code == 202:
    # Job was accepted (might fail later)
    job_id = response.json()['job_id']
    print(f"Job accepted: {job_id}")
    print("Waiting for it to fail...\n")
    
    time.sleep(2)
    
    response = requests.get(f"{API_URL}/api/v1/jobs/{job_id}")
    job_data = response.json()
    
    if job_data['status'] == 'failed':
        print("✅ Job failed as expected!")
        print(f"Error: {job_data.get('error')}")
else:
    print(f"Response: {response.text}")

## Test 10: Explore the Interactive Docs

The API has auto-generated interactive documentation!

In [ ]:
from IPython.display import IFrame

print("Opening interactive API documentation...\n")
print("You can also open in browser:")
print(f"  Swagger UI: {API_URL}/docs")
print(f"  ReDoc: {API_URL}/redoc")
print("\nThe interactive docs let you test all endpoints with a UI!")

# Display in notebook (if running in Jupyter)
IFrame(f"{API_URL}/docs", width=1000, height=600)

## ✅ API Testing Complete!

**What you tested:**
1. ✅ Health check endpoint
2. ✅ Version endpoint
3. ✅ POST /api/v1/flatten (async job submission)
4. ✅ GET /api/v1/jobs/{job_id} (job status polling)
5. ✅ POST /api/v1/compare (file comparison)
6. ✅ Error handling
7. ✅ Interactive API docs

**Next steps:**
- Explore the Swagger UI at http://localhost:8000/docs
- Try the `/extract` endpoint (commits to git)
- Integrate with your own workflow
- Read the code to understand how it all works!